In [54]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [28]:
pip install statsmodels

  Using cached statsmodels-0.14.5-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.5 kB)
  Using cached patsy-1.0.2-py2.py3-none-any.whl.metadata (3.6 kB)
Using cached statsmodels-0.14.5-cp313-cp313-macosx_11_0_arm64.whl (9.7 MB)
Using cached patsy-1.0.2-py2.py3-none-any.whl (233 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]
Note: you may need to restart the kernel to use updated packages.


In [85]:
df = pd.read_csv('../input/penn_jae.dat.txt', sep=' ')
df.head()


,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,Unnamed: 24,Unnamed: 25
0,10824,0,18,18,0,0,0,0,2,0,...,0,0,0,0,0,1,0,NaN,NaN,NaN
1,10635,2,7,3,0,0,0,0,0,0,...,1,0,0,0,1,0,0,NaN,NaN,NaN
2,10551,5,18,6,1,0,0,0,0,0,...,0,1,0,0,0,0,0,NaN,NaN,NaN
3,10824,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,NaN
4,10747,0,27,27,0,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,NaN


In [31]:
df = df[(df['tg'] == 0) | (df['tg'] == 4)]

In [32]:
df['T4'] = (df['tg'] == 4).astype(int)
df.head()

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,agelt35,agegt54,durable,nondurable,lusd,husd,muld,Unnamed: 24,Unnamed: 25,T4
0,10824,0,18,18,0,0,0,0,2,0,...,0,0,0,0,1,0,NaN,NaN,NaN,0
3,10824,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,NaN,NaN,NaN,0
4,10747,0,27,27,0,0,0,0,0,0,...,0,0,0,1,0,0,NaN,NaN,NaN,0
11,10607,4,9,9,0,0,0,0,0,0,...,0,0,0,0,0,1,NaN,NaN,NaN,1
12,10831,0,27,27,0,0,0,0,1,0,...,1,1,0,1,0,0,NaN,NaN,NaN,0


In [33]:
d = df['T4']

In [34]:
df['y'] = np.log(df['inuidur1'])

In [35]:
# Crear variables dummy para dep
dummies = pd.get_dummies(df['dep'], prefix='dep')

# Unirlas al dataframe original
df = pd.concat([df, dummies], axis=1)
df[['dep_0', 'dep_1', 'dep_2']] = df[['dep_0', 'dep_1', 'dep_2']].astype(int)

In [36]:
x = [
    'female', 'black', 'othrace',
    'dep_1', 'dep_2',
    'q2', 'q3', 'q4', 'q5', 'q6',
    'recall', 'agelt35', 'agegt54',
    'durable', 'nondurable', 'lusd', 'husd'
]

In [37]:
X = df[x]

In [38]:
y = df['y']

In [41]:
X.isnull().sum()

female        0
black         0
othrace       0
dep_1         0
dep_2         0
q2            0
q3            0
q4            0
q5            0
q6            0
recall        0
agelt35       0
agegt54       0
durable       0
nondurable    0
lusd          0
husd          0
dtype: int64

In [55]:
def dml(X, D, y, modely, modeld, nfolds=5):
    """
    Double Machine Learning for the Partially Linear Model setting with cross-fitting
    """
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    
    # Out-of-fold predictions (cross-fitting)
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)
    Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    
    # Residuals
    resy = y - yhat
    resD = D - Dhat
    
    # Final stage OLS on residuals
    df_res = pd.DataFrame({'resy': resy, 'resD': resD})
    ols = smf.ols('resy ~ resD', data=df_res).fit()
    
    return {
        'estimate': ols.params['resD'],
        'stderr': ols.bse['resD'],
        'rmse_y': np.sqrt(np.mean(resy**2)),
        'rmse_D': np.sqrt(np.mean(resD**2))
    }

In [43]:
# --- 4. Modelos para la primera etapa ---
model_y = make_pipeline(StandardScaler(), LassoCV(cv=5))
model_d = make_pipeline(StandardScaler(), LassoCV(cv=5))


In [ ]:
results = {}
# --- OLS ---
modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())
results['OLS C'] = dml(X, d, y, modely, modeld)

# --- LASSO ---
modely = make_pipeline(StandardScaler(), LassoCV(cv=5))
modeld = make_pipeline(StandardScaler(), LassoCV(cv=5))
results['Lasso'] = dml(X, d, y, modely, modeld)

# --- RANDOM FOREST ---
modely = make_pipeline(StandardScaler(),
                       RandomForestRegressor(n_estimators=200, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(),
                       RandomForestRegressor(n_estimators=200, min_samples_leaf=5, random_state=123))
results['Random Forest'] = dml(X, d, y, modely, modeld)

# --- NEURAL NETWORK ---
modely = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(100, 100),
                                    activation='relu', solver='adam',
                                    max_iter=500, early_stopping=True, random_state=123))
modeld = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(50, 50),
                                    activation='relu', solver='adam',
                                    max_iter=500, early_stopping=True, random_state=123))
results['Neural Net'] = dml(X, d, y, modely, modeld)

# --- NEURAL NETWORK nueva ---
modely = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(100, 100),
                                    activation='relu', solver='adam',
                                    max_iter=500, early_stopping=True, random_state=123))
modeld = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(50, 50),
                                    activation='relu', solver='adam',
                                    max_iter=500, early_stopping=True, random_state=123))

results['Neural Net (100x100 vs 50x50)'] = dml(X, d, y, modely, modeld)


In [74]:
table = pd.DataFrame(results).T
table['lower'] = table['estimate'] - 1.96*table['stderr']
table['upper'] = table['estimate'] + 1.96*table['stderr']
table = table[['estimate','stderr','lower','upper','rmse_y','rmse_D']]
table

,estimate,stderr,lower,upper,rmse_y,rmse_D
OLS,-0.066758,0.035178,-0.135707,0.002190,1.194866,0.475599
Lasso,-0.069715,0.035246,-0.138797,-0.000633,1.195114,0.474765
Random Forest,-0.073477,0.035305,-0.142676,-0.004279,1.217122,0.482672
Neural Net,-0.058134,0.034857,-0.126455,0.010186,1.203450,0.483519
Neural Net (100x100 vs 50x50),-0.058134,0.034857,-0.126455,0.010186,1.203450,0.483519


Los resultados muestran que el efecto estimado de D sobre Y es negativo (alrededor de −0.07) y estable entre modelos, lo que indica robustez.
Dado el RMSE ligeramente menor, seleccionamos el modelo Lasso como el más apropiado

In [76]:
def dml_no_cf(X, D, y, modely, modeld):
    # Ajuste en TODO el conjunto (sin out-of-fold)
    my = modely.fit(X, y)
    md = modeld.fit(X, D)

    yhat = my.predict(X)
    Dhat = md.predict(X)

    resy = y - yhat
    resD = D - Dhat

    df = pd.DataFrame({'resy': resy, 'resD': resD})
    ols = smf.ols('resy ~ resD', data=df).fit()

    out = {
        'estimate': ols.params['resD'],
        'stderr': ols.bse['resD'],
        'rmse_y': float(np.sqrt(np.mean(resy**2))),
        'rmse_D': float(np.sqrt(np.mean(resD**2))),
    }
    return out

In [82]:
results_nocf = {}

# --- OLS (lineal) ---
modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())
results_nocf['OLS'] = dml_no_cf(X, d, y, modely, modeld)

# --- Lasso ---
modely = make_pipeline(StandardScaler(), LassoCV(cv=5, random_state=123))
modeld = make_pipeline(StandardScaler(), LassoCV(cv=5, random_state=123))
results_nocf['Lasso'] = dml_no_cf(X, d, y, modely, modeld)

# --- Random Forest ---
modely = make_pipeline(StandardScaler(),
                       RandomForestRegressor(n_estimators=200, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(),
                       RandomForestRegressor(n_estimators=200, min_samples_leaf=5, random_state=123))
results_nocf['Random Forest'] = dml_no_cf(X, d, y, modely, modeld)

# --- Neural Net ---
modely = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(100,100), activation='relu',
                                    solver='adam', max_iter=500, early_stopping=True, random_state=123))
modeld = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(50,50), activation='relu',
                                    solver='adam', max_iter=500, early_stopping=True, random_state=123))
results_nocf['Neural Net'] = dml_no_cf(X, d, y, modely, modeld)

# Tabla como en los labs
table_nocf = pd.DataFrame(results_nocf).T
table_nocf['lower'] = table_nocf['estimate'] - 1.96*table_nocf['stderr']
table_nocf['upper'] = table_nocf['estimate'] + 1.96*table_nocf['stderr']
table_nocf = table_nocf[['estimate','stderr','lower','upper','rmse_y','rmse_D']]
table_nocf

,estimate,stderr,lower,upper,rmse_y,rmse_D
OLS,-0.071216,0.035186,-0.140181,-0.002251,1.189571,0.473352
Lasso,-0.071227,0.035151,-0.140123,-0.002330,1.189576,0.473824
Random Forest,-0.074873,0.035200,-0.143865,-0.005880,1.125760,0.447765
Neural Net,-0.068629,0.035202,-0.137625,0.000368,1.159498,0.461366


In [84]:
rmse_summary = pd.DataFrame({
    'RMSE_Y (CF)': table['rmse_y'],
    'RMSE_Y (no CF)': table_nocf['rmse_y'],
    'RMSE_D (CF)': table['rmse_D'],
    'RMSE_D (no CF)': table_nocf['rmse_D']
})

rmse_summary = rmse_summary.loc[['OLS', 'Lasso', 'Random Forest', 'Neural Net']]
rmse_summary

,RMSE_Y (CF),RMSE_Y (no CF),RMSE_D (CF),RMSE_D (no CF)
OLS,1.194866,1.189571,0.475599,0.473352
Lasso,1.195114,1.189576,0.474765,0.473824
Random Forest,1.217122,1.125760,0.482672,0.447765
Neural Net,1.203450,1.159498,0.483519,0.461366


#### What can you say about the RMSE for predicting y and d?

Los resultados muestran que los RMSE son menores cuando no se usa cross-fitting, tanto para Y como para D. Esto ocurre porque en ese caso los modelos de machine learning predicen sobre los mismos datos con los que fueron entrenados. Es decir, el modelo “ya conoce” la información y por eso predice mejor dentro de la muestra, en cambio, con cross-fitting las predicciones se hacen sobre datos que el modelo no ha visto, lo que refleja un desempeño real fuera de muestra y por eso los errores son mayores, aunque más honestos.

#### Why is it that estimating with one function yields lower RMSE than another?

Sin cross-fitting, los modelos aprenden todas las relaciones (incluso las espurias) entre D y X, y al hacerlo mezclan el efecto causal de D con la parte explicada por X, es decir que algoritmo se enfoca en predecir bien Y, no en aislar el efecto causal de d, por lo que los errores aparentan ser más bajos, pero el estimador B queda sesgado, entonces el cross-fitting soluciona esto separando el paso de entrenar del de estimar, garantizando que las predicciones no dependan de los mismos datos usados en la regresión final, así se mantiene la ortogonalidad de Neyman, que protege el β frente a pequeños errores en los modelos auxiliares.

#### What problem would we have if we chose to estimate without cross-fitting?

Si no se usa cross-fitting se rompe la independencia entre las predicciones y la estimación final, lo que genera sesgo en B y errores estándar subestimados, el modelo se “ajusta demasiado” y las inferencias dejan de ser válidas, ya que el estimador deja de cumplir la condición de Neyman orthogonality, en otras palabras, el RMSE baja artificialmente parece que el modelo mejora, pero en realidad se pierde la interpretación causal correcta del parámetro.